# 数据预处理

In [1]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pylab

%matplotlib inline

## 读取数据
- data_all是Lending Club 2015年的借贷数据

In [3]:
raw_df = pd.read_csv('../data/data_all.csv')
print '数据规模:',raw_df.shape

数据规模: (421095, 85)


In [4]:
raw_df.head()

,id,member_id,loan_amnt,term,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68444620,73334399,35000,60 months,Foreign Service Officer,10+ years,MORTGAGE,128000.0,Source Verified,Dec-2015,...,0,1,91.3,33.3,0,0,220950,49644,26800,71700
1,68426545,73316365,16000,60 months,Senior Structural Designer,1 year,MORTGAGE,70000.0,Not Verified,Dec-2015,...,0,1,100.0,60.0,0,0,309638,62407,45900,45838
2,68466926,73356763,10000,36 months,SERVICE MANAGER,6 years,RENT,85000.0,Not Verified,Dec-2015,...,0,2,95.7,28.6,1,0,61099,27957,16400,30799
3,68356614,73246437,7200,36 months,Legal Administrative Assistant,10+ years,MORTGAGE,72500.0,Verified,Dec-2015,...,0,3,100.0,0.0,0,0,189950,20256,15500,22450
4,68387134,73276991,2500,36 months,Teacher,10+ years,MORTGAGE,50000.0,Verified,Dec-2015,...,1,3,68.4,0.0,0,0,196429,197169,700,193929


## 删除如下的特征
- 非数值型特征
- 对识别欺诈无意义的特征：id, member_id

In [5]:
reamin_features =[feat for feat in raw_df.select_dtypes(include=['float64', 'int64']).keys() \
                  if feat not in ['id', 'loan_status', 'member_id','issue_d']]
feature_df = raw_df[reamin_features]
df = raw_df.copy()
df = df[reamin_features+['loan_status','issue_d']]

## 将月份转化为数值

In [6]:
def map_month(x):
    """ Map the month strings to integers.
    """
    if x!=x:
        return 0
    if "Jan" in x:
        return 1
    if "Apr" in x:
        return 4
    if 'Aug' in x:
        return 8
    if 'Dec' in x:
        return 12
    if 'Feb' in x:
        return 2
    if 'Jul' in x:
        return 7
    if 'Jun' in x:
        return 6
    if 'Mar' in x:
        return 3
    if 'May' in x:
         return 5
    if 'Nov' in x:
        return 11
    if 'Oct' in x:
        return 10
    if 'Sep' in x:
        return 9

In [7]:
df.issue_d = map(map_month,df.issue_d)
month_max = max(df.issue_d)
print '一共'+str(month_max)+'个月的数据'

一共12个月的数据


## 统计各特征的缺失比例
- 这里不包括计算KS、IV、PSI的标签数据: loan_status, issue_d

In [8]:
def get_nan_cnt(feature_df):
    """feature_df is a data frame.
       return the missing value counts of every feature.
    """
    nan_cnt = []
    nan_cnt =  (feature_df!=feature_df).sum(axis=0)
    return nan_cnt

In [9]:
nan_cnt = get_nan_cnt(feature_df)
total = raw_df.shape[0]
nan_cnt = nan_cnt *1.0 / total
nan_df = pd.DataFrame(nan_cnt,columns=['nan_ratio'])
nan_df.index.name = 'feature'
print '缺失比例最高的一些特征：'
nan_df.sort_values(by='nan_ratio',ascending=False).head(20)

缺失比例最高的一些特征：


,nan_ratio
feature,
il_util,0.955789
mths_since_rcnt_il,0.950581
open_acc_6m,0.949247
inq_last_12m,0.949247
total_cu_tl,0.949247
inq_fi,0.949247
all_util,0.949247
max_bal_bc,0.949247
open_rv_24m,0.949247


## 输出缺失比例和处理后的数据

In [10]:
df.to_csv('../data/data_clean.csv',index=False)
nan_df.to_csv('../output/Completeness.csv')